In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
class PetraM_Helper:
    def __init__(self):
        self.species_list = [] # list for the names of species
        self.species_mass = {}  # dictunary with corrisponding species mass [kg]
        self.species_charge = {} # dictunary with corrisponding species charge [C]
        self.species_rho = {} # species rho profile for the following density and temp profiles.  
        self.species_density = {}  # species density over x in m^-3
        self.species_temp = {}     # species temp over x
        self.epsi0 = 8.85418782e-12 # permitivity of free space
        self.clight = 3e8 # m/s, light speed in vac

    def set_species(self, name, mass, charge, density, temp, rho, ndisti):
        """
        name: A name for the species. 'D', 'e', etc
        mass: species mass in kg
        charge: species charge in C
        density: profile over rho  in m^-3
        temp: profile over rho in KeV 
        rho: rho profile of type self.rho_type for the density and temp profiles
        ndisti: 0 for maxwellian treatment.  1 for non-maxwellian -> requires cql3d file. 
        """
        self.species_list.append(name)
        self.species_mass[name] = mass
        self.species_charge[name] = charge
        self.species_density[name] = density
        self.species_temp[name] = temp 
        self.species_rho[name] = rho
        self.species_ndist[name] = ndisti

        if self.species_list[0] != 'e' and self.species_list[0] != 'E':
            raise ValueError(f'The first species in self.species_list must be electrons, not "{self.species_list[0]}"') 

    def get_R(self, w, B, n):
        R = 1
        for name in self.species_list:
            q = self.species_charge[name]
            mass = self.species_mass[name]
            Omega = q*B/mass
            wp_sq = n*q**2/(self.epsi0*mass)
            R = R - wp_sq/(w*(w-Omega))
        return R

    def get_L(self, w, B, n):
        L = 1
        for name in self.species_list:
            q = self.species_charge[name]
            mass = self.species_mass[name]
            Omega = q*B/mass
            wp_sq = n*q**2/(self.epsi0*mass)
            L = L - wp_sq/(w*(w+Omega))
        return L

    def get_S(self, w, B, n):
        R = self.get_R(w, B, n)
        L = self.get_L(w, B ,n)
        return 0.5*(R+L)

    def get_D(self, w, B, n):
        R = self.get_R(w, B, n)
        L = self.get_L(w, B ,n)
        return 0.5*(R-L) 

    def get_P(self, w, n):
        P = 1
        for name in self.species_list:
            q = self.species_charge[name]
            mass = self.species_mass[name]
            wp_sq = n*q**2/(self.epsi0*mass)
            P = P - wp_sq/w**2
        return P    

    def fast_wave_n_perp(self, n_par, w, B, n):  # STIX eq 2-19 
        R = self.get_R(w, B, n)  
        L = self.get_L(w, B, n)
        S = self.get_S(w, B, n)   
        return (R - n_par**2)*(L-n_par**2) / (S - n_par**2)   

    def slow_wave_n_perp(self, n_par, w, B, n):  # from plasma phys. control. fusion 59 2017 054002 J Ongene Recent advances in phys and tech of ICRH
        R = self.get_R(w, B, n)  
        L = self.get_L(w, B, n)
        S = self.get_S(w, B, n)
        P = self.get_P(w, B, n) 
        k0 = w/self.clight
        k_par = n_par*k0
        return P*(k0**2*S - k_par**2)/S # this is n_perp^2 . 


'''
for a given density profile, you can plot 
n_perp^2 for fast and slow wave
the R and L cutoff density location 
The S resonant location: S = 0 and S = n_par^2 
'''